In [1]:
import pandas as pd 
import math

In [2]:
def tran_to_list(df):

    df = df.sort_values(by='TransactionNo', ascending=False)
    prev=-1
    list= []

    trans=[]
    for _, row in df.iterrows():
        if(row["TransactionNo"]!=prev and prev!=-1):
            trans.append(list)
            list=[]
            list.append(row["Items"])
            prev=row["TransactionNo"]
            
        if not (contains(list,[row["Items"]])):
            list.append(row["Items"])
            prev=row["TransactionNo"]
    
    trans.append(list)

    return trans

In [3]:
def generate_combinations(arr, r):
    """
    Generates combinations of length 'r' from the given array 'arr'
    """
    n = len(arr)
    # Base case: if r is 0 or greater than length of arr
    if r == 0 or r > n:
        return [[]]
    # Base case: if r is equal to length of arr
    if r == n:
        return [arr]

    # Recursive case
    combinations = []

    # Generate combinations including the first element
    for comb in generate_combinations(arr[1:], r - 1):
        combinations.append([arr[0]] + comb)

    # Generate combinations excluding the first element
    combinations.extend(generate_combinations(arr[1:], r))

    return combinations

In [4]:
def contains(larger, smaller):
    
    for i in smaller:
        flag=0
        for k in larger:
            if(i==k):
                flag=1
                break
        if(flag==0):
            return False
    return True
            


In [5]:
#combination of l 
def com_PL(L_initial,prev_to_remove,num):
    l1=[]
    for i in range(len(L_initial)):
        for j in range(i+1,len(L_initial)):
            list=[]
            all=[]
            list.extend(L_initial[i])
            list.extend(L_initial[j])
            unique_list = []
            
            # Iterate over the original list
            for item in list:
                # Check if the item is not already in the unique_list
                if item not in unique_list:
                    # Append the item to the unique_list
                    unique_list.append(item)
            
            all = generate_combinations(unique_list,num)
            for com in all:
                add = 1 
                #search in the non frequent set
                for k in prev_to_remove:
                    if(contains(com,k)):
                        add=0
                        break

                if add:
                    for kok in l1:
                            if(contains(kok,com)):
                                add=0
                                break
                if (add):   
                    l1.extend([com])
    return l1

In [6]:
# count for Ls
def trans_count(combinations,trans,msl):
    to_remove=[]
    L_initial=[]
    L_initial_pair=[]
    flag=0
    for com in combinations :
        count=0
        #print(com)
        for row in trans:
            if(len(com)<=len(row)):
                if contains(row,com):
                    count+=1
    
        if(count>=msl):
            L_initial_pair.append([com, count])
            #print([com, count])
            L_initial.append(com)
            flag=1
        else:
            #for the next L
            to_remove.append(com)
            
    return to_remove,L_initial,L_initial_pair,flag

In [7]:
def frequentItemsSets(unique_values_per_column,trans,ms):
    Ls=[]
    count=1
    flag =1
    prev_l=unique_values_per_column
    prev_to_remove=[]
    while(flag):
        prev_l =com_PL(prev_l,prev_to_remove,count)
        prev_to_remove,prev_l,L,flag =trans_count(prev_l,trans,ms)
        if(flag):
            Ls.append(L)
            count+=1
    return Ls

In [8]:
def confidence(freqsets, mcl):
    all_conf = []
    for i in range(1, len(freqsets)):
        current = freqsets[i]
        #loop over each item
        for pair in current:
            list = pair[0]
            count = pair[1]
            bcount = len(pair[0]) - 1
            for fcount in range(1, len(list)):
                if (fcount > bcount):
                    break
                combs = generate_combinations(list, fcount)
                for comb in combs:
                    for pp in freqsets[len(comb) - 1]:
                        if (contains(comb, pp[0])):
                            conf = count / pp[1]
                            if (conf >= mcl):
                                difference = [item for item in list if item not in pp[0]]
                                all_conf.append([pp[0], difference, round(conf, 2)])
                            break
    return all_conf

In [9]:
def apriori(msl,mcl,data):

    # combining all the transactions into a list of lists 
    tran= tran_to_list(data)

    unique_values_per_column =data["Items"].unique().tolist()
    for i in range(len(unique_values_per_column)):
        unique_values_per_column[i]= [unique_values_per_column[i]]

    ms= msl
    print(ms)
    
    Freqentsets = frequentItemsSets(unique_values_per_column,tran,ms)

    all_conff=confidence(Freqentsets,mcl)
        
    
    return all_conff,Freqentsets

In [10]:
data = pd.read_csv("/Users/faressaad/Downloads/Assignment1/Bakery.csv")
data.drop(columns=["DateTime"],inplace=True)
data.head(5)

,TransactionNo,Items,Daypart,DayType
0,1,Bread,Morning,Weekend
1,2,Scandinavian,Morning,Weekend
2,2,Scandinavian,Morning,Weekend
3,3,Hot chocolate,Morning,Weekend
4,3,Jam,Morning,Weekend


In [11]:
print(tran_to_list(data[:100]))

[['Coffee'], ['Jam', 'Muffin', 'Bread', 'Juice', "Ella's Kitchen Pouches"], ['Coffee'], ['Medialuna', 'Hot chocolate', 'Coffee'], ['Coffee', 'Medialuna'], ['Fudge', 'Scandinavian'], ['Tea', 'Bread'], ['Scandinavian'], ['Muffin', 'Coffee', 'Tartine', 'Juice'], ['Scandinavian'], ['Tea', 'Bread'], ['Bread', 'Coffee'], ['Bread', 'Tea'], ['Scandinavian', 'Muffin'], ['Muffin', 'Coffee'], ['Coffee'], ['Scandinavian', 'Muffin'], ['Basket'], ['Bread'], ['Jam', 'Bread'], ['Bread', 'Coffee'], ['Scandinavian'], ['Fudge'], ['Scandinavian'], ['Bread'], ['Scandinavian', 'Muffin'], ['Jam'], ['Bread', 'Medialuna', 'Coffee'], ['Medialuna', 'Bread'], ['Bread', 'Farm House'], ['Farm House'], ['Hot chocolate'], ['Coffee', 'Medialuna', 'Bread'], ['Scandinavian', 'Mineral water'], ['Pastry', 'Medialuna', 'Bread'], ['Coffee', 'Bread', 'Basket'], ['Tea', 'Pastry', 'Tartine', 'Coffee', 'Jam'], ['Medialuna', 'Bread'], ['Scandinavian', 'Medialuna'], ['Muffin', 'Bread'], ['Bread', 'Pastry'], ['Medialuna', 'Coffee'

In [15]:
con,feq=apriori(2,.5,data[:100])

2


In [16]:
print(feq)

[[[['Bread'], 20], [['Scandinavian'], 11], [['Hot chocolate'], 3], [['Jam'], 5], [['Muffin'], 9], [['Coffee'], 15], [['Pastry'], 6], [['Medialuna'], 10], [['Tea'], 5], [['Tartine'], 2], [['Basket'], 2], [['Farm House'], 2], [['Fudge'], 2], [['Juice'], 2]], [[['Bread', 'Jam'], 2], [['Bread', 'Muffin'], 2], [['Bread', 'Coffee'], 6], [['Bread', 'Pastry'], 3], [['Bread', 'Medialuna'], 5], [['Bread', 'Tea'], 3], [['Scandinavian', 'Muffin'], 3], [['Muffin', 'Coffee'], 2], [['Muffin', 'Juice'], 2], [['Coffee', 'Pastry'], 3], [['Coffee', 'Medialuna'], 5], [['Coffee', 'Tea'], 2], [['Coffee', 'Tartine'], 2], [['Pastry', 'Medialuna'], 3], [['Pastry', 'Tea'], 2]], [[['Bread', 'Coffee', 'Medialuna'], 2], [['Coffee', 'Pastry', 'Tea'], 2]]]


In [18]:
print(con)

[[['Pastry'], ['Bread'], 0.5], [['Medialuna'], ['Bread'], 0.5], [['Tea'], ['Bread'], 0.6], [['Juice'], ['Muffin'], 1.0], [['Pastry'], ['Coffee'], 0.5], [['Medialuna'], ['Coffee'], 0.5], [['Tartine'], ['Coffee'], 1.0], [['Pastry'], ['Medialuna'], 0.5], [['Coffee', 'Pastry'], ['Tea'], 0.67], [['Coffee', 'Tea'], ['Pastry'], 1.0], [['Pastry', 'Tea'], ['Coffee'], 1.0]]
